In [ ]:
df_source = spark.read.format("delta").load("Tables/dbo/accounts")
df_source.createOrReplaceTempView("source_view")
src_path="Tables/dbo/accounts"

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 11, Finished, Available, Finished)

In [ ]:
create_table_query = """
CREATE TABLE Account_SCD (
    account_id int,
    customer_id int,
    account_type string,
    balance float,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Tables/Gold_layer/Account_SCD'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 8, Finished, Available, Finished)

DataFrame[]

In [ ]:
from delta.tables import DeltaTable

target_path = "Tables/Gold_layer/Account_SCD"  
delta_target = DeltaTable.forPath(spark, target_path)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 9, Finished, Available, Finished)

In [ ]:
from pyspark.sql.functions import *
df_src1= df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fa875736-2b6b-4536-80c3-ad43aeedc38a)

In [ ]:
df_src1=df_src1.alias("src").join(delta_target.toDF().alias("tgt"),((col("src.account_id")==col("tgt.account_id"))&(col("src.hash_key")==col("tgt.hash_key"))),"anti").select(col("src.*"))
df_src1.show()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 14, Finished, Available, Finished)

+----------+-----------+------------+-------+----------+
|account_id|customer_id|account_type|balance|  hash_key|
+----------+-----------+------------+-------+----------+
|         1|         45|     Savings| 1000.5|2454403084|
|         2|         12|    Checking|2500.75| 796571617|
|         3|         78|     Savings| 1500.0| 199654578|
|         4|         34|    Checking|3000.25| 761699333|
|         5|         56|     Savings|  500.0|3886787448|
|         6|         23|    Checking| 1200.5| 589336936|
|         7|         89|     Savings| 800.75|  74260363|
|         8|         67|    Checking| 2200.0|3401041467|
|         9|         14|     Savings| 900.25|1397533172|
|        10|         92|    Checking| 1800.5| 533177005|
|        11|          3|     Savings|1100.75| 744264830|
|        12|         81|    Checking| 2700.0|   4249714|
|        13|         29|     Savings|1300.25|3025925162|
|        14|         64|    Checking| 3200.5|3349647104|
|        15|         47|     Sa

In [ ]:
from pyspark.sql.functions import col

delta_target.alias("tgt").merge(df_src1.alias("src"),"tgt.account_id = src.account_id")\
    .whenMatchedUpdate(set={"tgt.account_id":"src.account_id","tgt.customer_id":"src.customer_id","tgt.account_type":"src.account_type","tgt.balance":"src.balance","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.account_id":"src.account_id","tgt.customer_id":"src.customer_id","tgt.account_type":"src.account_type","tgt.balance":"src.balance","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
display(spark.read.format("delta").option("header","true").load(target_path))

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5e1f4021-03d8-4ffe-a09b-141f8f528ff0)

Customer table

In [ ]:
create_table_query = """
CREATE TABLE Customer_SCD1 (
    customer_id int,
    first_name string,
    last_name string,
    address string,
    city string,
    state string,
    zip string,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Tables/Gold_layer/Customer_SCD1'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 24, Finished, Available, Finished)

DataFrame[]

In [ ]:
from pyspark.sql.functions import *
df_source = spark.read.format("delta").load("Tables/dbo/customers")
df_source.createOrReplaceTempView("source_view")
src_path="Tables/dbo/customers"
print(src_path)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 18, Finished, Available, Finished)

Tables/dbo/customers


In [ ]:
df_src1=df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ef8131c5-f6f9-4aae-9708-13ca302e58f8)

In [ ]:
from delta.tables import DeltaTable
tgt_path = "Tables/Gold_layer/Customer_SCD1"  
dbtable = DeltaTable.forPath(spark, tgt_path)
dbtable.toDF().show()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 26, Finished, Available, Finished)

+-----------+----------+---------+-------+----+-----+---+--------+----------+------------+----------+------------+
|customer_id|first_name|last_name|address|city|state|zip|hash_key|created_by|created_date|updated_by|updated_date|
+-----------+----------+---------+-------+----+-----+---+--------+----------+------------+----------+------------+
+-----------+----------+---------+-------+----+-----+---+--------+----------+------------+----------+------------+



In [ ]:
df_src1=df_src1.alias("src").join(dbtable.toDF().alias("tgt"), ((col("src.customer_id") == col("tgt.customer_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_src1.show()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 27, Finished, Available, Finished)

+-----------+-----------+---------+---------------+-------------+-----+------+----------+
|customer_id| first_name|last_name|        address|         city|state|   zip|  hash_key|
+-----------+-----------+---------+---------------+-------------+-----+------+----------+
|          1|       John|      Doe|     123 Elm St|      Toronto|   ON|M4B1B3|2373314683|
|          2|       Jane|    Smith|  456 Maple Ave|       Ottawa|   ON|K1A0B1|3911254336|
|          3|    Michael|  Johnson|     789 Oak Dr|     Montreal|   QC|H1A1A1|4114944968|
|          4|      Emily|    Davis|    101 Pine Rd|      Calgary|   AB|T2A0A1| 165372536|
|          5|      David|   Wilson| 202 Birch Blvd|    Vancouver|   BC|V5K0A1|3284019540|
|          6|       Emma|    Clark|   505 Cedar St|      Halifax|   NS|B3H0A1|4268191859|
|          7|      James| Martinez|  606 Spruce Ln|     Winnipeg|   MB|R3C0A1|4278935663|
|          8|     Olivia|   Garcia|     707 Fir St|     Edmonton|   AB|T5A0A1| 443479193|
|         

In [ ]:
dbtable.alias("tgt").merge(df_src1.alias("src"),"tgt.customer_id = src.customer_id")\
    .whenMatchedUpdate(set={"tgt.customer_id":"src.customer_id","tgt.first_name":"src.first_name","tgt.last_name":"src.last_name","tgt.address":"src.address","tgt.city":"src.city","tgt.state":"src.state","tgt.zip":"src.zip","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.customer_id":"src.customer_id","tgt.first_name":"src.first_name","tgt.last_name":"src.last_name","tgt.address":"src.address","tgt.city":"src.city","tgt.state":"src.state","tgt.zip":"src.zip","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 28, Finished, Available, Finished)

In [ ]:
display(spark.read.format("delta").option("header","true").load(tgt_path))

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 29, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 47a9984a-6951-4084-8e3d-f8d4245715c7)

Loan_Payment

In [ ]:
create_table_query = """
CREATE TABLE Loan_Payment (
  payment_id int,
  loan_id int,
  payment_date timestamp,
  payment_amount float,
  hash_key bigint,
  created_by string,
  created_date timestamp,
  updated_by string,
  updated_date timestamp
)
USING DELTA
LOCATION 'Tables/Gold_layer/Loan_Payment'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 30, Finished, Available, Finished)

DataFrame[]

In [ ]:
from pyspark.sql.functions import *
df_source = spark.read.format("delta").load("Tables/dbo/loan_payments")
df_source.createOrReplaceTempView("source_view")
src_path="Tables/dbo/loan_payments"
print(src_path)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 34, Finished, Available, Finished)

Tables/dbo/loan_payments


In [ ]:
df_src1=df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 35, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0ff5d323-0102-4913-9c70-e2b0a35c6463)

In [ ]:
from delta.tables import DeltaTable
tgt_path = "Tables/Gold_layer/Loan_Payment"  
dbtable = DeltaTable.forPath(spark, tgt_path)
dbtable.toDF().show()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 36, Finished, Available, Finished)

+----------+-------+------------+--------------+--------+----------+------------+----------+------------+
|payment_id|loan_id|payment_date|payment_amount|hash_key|created_by|created_date|updated_by|updated_date|
+----------+-------+------------+--------------+--------+----------+------------+----------+------------+
+----------+-------+------------+--------------+--------+----------+------------+----------+------------+



In [ ]:
df_src1=df_src1.alias("src").join(dbtable.toDF().alias("tgt"), ((col("src.payment_id") == col("tgt.payment_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_src1.show()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 37, Finished, Available, Finished)

+----------+-------+-------------------+--------------+----------+
|payment_id|loan_id|       payment_date|payment_amount|  hash_key|
+----------+-------+-------------------+--------------+----------+
|         1|     45|2024-01-01 00:00:00|         100.0|2281099843|
|         2|     23|2024-01-02 00:00:00|         150.0|1187304668|
|         3|     67|2024-01-03 00:00:00|         200.0| 820265650|
|         4|     89|2024-01-04 00:00:00|         250.0|1674179584|
|         5|     12|2024-01-05 00:00:00|         300.0| 366101476|
|         6|     34|2024-01-06 00:00:00|         350.0|4269069221|
|         7|     56|2024-01-07 00:00:00|         400.0|  54904002|
|         8|     78|2024-01-08 00:00:00|         450.0| 585028383|
|         9|     90|2024-01-09 00:00:00|         500.0|3119229326|
|        10|     11|2024-01-10 00:00:00|         550.0|3168535820|
|        11|     22|2024-01-11 00:00:00|         600.0|3698267372|
|        12|     33|2024-01-12 00:00:00|         650.0| 198088

In [ ]:
dbtable.alias("tgt").merge(df_src1.alias("src"),"tgt.payment_id = src.payment_id")\
    .whenMatchedUpdate(set={"tgt.payment_id":"src.payment_id","tgt.loan_id":"src.loan_id","tgt.payment_date":"src.payment_date","tgt.payment_amount":"src.payment_amount","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.payment_id":"src.payment_id","tgt.loan_id":"src.loan_id","tgt.payment_date":"src.payment_date","tgt.payment_amount":"src.payment_amount","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 38, Finished, Available, Finished)

In [ ]:
display(spark.read.format("delta").option("header","true").load(tgt_path))

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 39, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2907c44a-bdd7-4ad1-bc9f-fa2dcadb4f7d)

loans

In [ ]:
create_table_query = """
CREATE TABLE Loans1 (
  loan_id int,
  customer_id int,
  interest_rate float,
  loan_term int,
  loan_amount float,
  hash_key bigint,
  created_by string,
  created_date timestamp,
  updated_by string,
  updated_date timestamp
)
USING DELTA
LOCATION 'Tables/Gold_layer/Loans1'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 42, Finished, Available, Finished)

DataFrame[]

In [ ]:
from pyspark.sql.functions import *
df_source = spark.read.format("delta").load("Tables/dbo/loans")
df_source.createOrReplaceTempView("source_view")
src_path="Tables/dbo/loans"
print(src_path)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 43, Finished, Available, Finished)

Tables/dbo/loans


In [ ]:
df_src1=df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 44, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 84290ebf-c96b-4f9e-96c4-dc7ae16fe02a)

In [ ]:
from delta.tables import DeltaTable
tgt_path = "Tables/Gold_layer/Loans1"  
dbtable = DeltaTable.forPath(spark, tgt_path)
dbtable.toDF().show()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 45, Finished, Available, Finished)

+-------+-----------+-------------+---------+-----------+--------+----------+------------+----------+------------+
|loan_id|customer_id|interest_rate|loan_term|loan_amount|hash_key|created_by|created_date|updated_by|updated_date|
+-------+-----------+-------------+---------+-----------+--------+----------+------------+----------+------------+
+-------+-----------+-------------+---------+-----------+--------+----------+------------+----------+------------+



In [ ]:
df_ln1=df_src1.alias("src").join(dbtable.toDF().alias("tgt"), ((col("src.loan_id") == col("tgt.loan_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_ln1.show()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 47, Finished, Available, Finished)

+-------+-----------+-----------+-------------+---------+----------+
|loan_id|customer_id|loan_amount|interest_rate|loan_term|  hash_key|
+-------+-----------+-----------+-------------+---------+----------+
|      1|         45|    10000.5|          5.5|       36|2502041691|
|      2|         12|   20000.75|          4.5|       48|3355992476|
|      3|         78|    15000.0|          6.0|       60|4060872917|
|      4|         34|   30000.25|          3.5|       24|2847880128|
|      5|         56|    25000.0|          5.0|       36|1867834730|
|      6|         23|    17500.5|          4.0|       48| 172242294|
|      7|         89|   22500.75|          6.5|       60|1348082437|
|      8|         67|    27500.0|          3.0|       24|4106075248|
|      9|         14|   32500.25|          5.5|       36|3981072152|
|     10|         92|    37500.5|          4.5|       48| 888718168|
|     11|          3|   10000.75|          6.0|       60|2179146733|
|     12|         81|    20000.0| 

In [ ]:
dbtable.alias("tgt").merge(df_ln1.alias("src"),"tgt.loan_id = src.loan_id")\
    .whenMatchedUpdate(set={"tgt.loan_id":"src.loan_id","tgt.customer_id":"src.customer_id","tgt.loan_amount":"src.loan_amount","tgt.interest_rate":"src.interest_rate","tgt.loan_term":"src.loan_term","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.loan_id":"src.loan_id","tgt.customer_id":"src.customer_id","tgt.loan_amount":"src.loan_amount","tgt.interest_rate":"src.interest_rate","tgt.loan_term":"src.loan_term","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 48, Finished, Available, Finished)

In [ ]:
display(spark.read.format("delta").option("header","true").load(tgt_path))

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 49, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e520ee04-2700-47d7-86e6-ae90908dac81)

Transaction

In [67]:
create_table_query = """
CREATE TABLE Transactions1 (
  transaction_id int,
  account_id int,
  transaction_date timestamp,
  transaction_amount float,
  transaction_type string,
  hash_key bigint,
  created_by string,
  created_date timestamp,
  updated_by string,
  updated_date timestamp
)
USING DELTA
LOCATION 'Tables/Gold_layer/Transactions1'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 69, Finished, Available, Finished)

DataFrame[]

In [ ]:
from pyspark.sql.functions import *
df_source = spark.read.format("delta").load("Tables/dbo/transactions")
df_source.createOrReplaceTempView("source_view")
src_path="Tables/dbo/transactions"
print(src_path)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 61, Finished, Available, Finished)

Tables/dbo/transactions


In [ ]:
df_src1=df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 60, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 76e83595-93e5-42de-8d6f-2a14137200a2)

In [68]:
from delta.tables import DeltaTable
tgt_path = "Tables/Gold_layer/Transactions1"  
dbtable = DeltaTable.forPath(spark, tgt_path)
dbtable.toDF().show()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 70, Finished, Available, Finished)

+--------------+----------+----------------+------------------+----------------+--------+----------+------------+----------+------------+
|transaction_id|account_id|transaction_date|transaction_amount|transaction_type|hash_key|created_by|created_date|updated_by|updated_date|
+--------------+----------+----------------+------------------+----------------+--------+----------+------------+----------+------------+
+--------------+----------+----------------+------------------+----------------+--------+----------+------------+----------+------------+



In [69]:
df_ts1=df_src1.alias("src").join(dbtable.toDF().alias("tgt"), ((col("src.transaction_id") == col("tgt.transaction_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_ts1.show()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 71, Finished, Available, Finished)

+--------------+----------+-------------------+------------------+----------------+----------+
|transaction_id|account_id|   transaction_date|transaction_amount|transaction_type|  hash_key|
+--------------+----------+-------------------+------------------+----------------+----------+
|             1|        45|2024-01-01 00:00:00|             100.5|         Deposit|2406044542|
|             2|        12|2024-01-02 00:00:00|            200.75|      Withdrawal| 199279155|
|             3|        78|2024-01-03 00:00:00|             150.0|         Deposit|3923312757|
|             4|        34|2024-01-04 00:00:00|            300.25|      Withdrawal|2322867790|
|             5|        56|2024-01-05 00:00:00|             250.0|         Deposit|4026887228|
|             6|        23|2024-01-06 00:00:00|             175.0|      Withdrawal|2653824275|
|             7|        89|2024-01-07 00:00:00|             225.5|         Deposit|3405751124|
|             8|        67|2024-01-08 00:00:00|   

In [70]:
dbtable.alias("tgt").merge(df_ts1.alias("src"),"tgt.transaction_id = src.transaction_id")\
    .whenMatchedUpdate(set={"tgt.transaction_id":"src.transaction_id","tgt.account_id":"src.account_id","tgt.transaction_date":"src.transaction_date","tgt.transaction_amount":"src.transaction_amount","tgt.transaction_type":"src.transaction_type","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.transaction_id":"src.transaction_id","tgt.account_id":"src.account_id","tgt.transaction_date":"src.transaction_date","tgt.transaction_amount":"src.transaction_amount","tgt.transaction_type":"src.transaction_type","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 72, Finished, Available, Finished)

In [71]:
display(spark.read.format("delta").option("header","true").load(tgt_path))

StatementMeta(, 562ad1e9-61d0-4b99-b5a0-948cef32d6c2, 73, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4277816d-7d3d-47d6-a9b3-4e009280aae5)